im about to loose it all

In [1]:
from server_functions import ServerFunctions
import simulator as sim
import client.model as mod


In [2]:
datafiles = (
    "/Users/Axel/fedn/examples/server-functions/new_data/B0005_part1.csv",
    "/Users/Axel/fedn/examples/server-functions/new_data/B0005_part2.csv",
    "/Users/Axel/fedn/examples/server-functions/new_data/B0006_part1.csv",
    "/Users/Axel/fedn/examples/server-functions/new_data/B0006_part2.csv",
    "/Users/Axel/fedn/examples/server-functions/new_data/B0007_part1.csv",
    "/Users/Axel/fedn/examples/server-functions/new_data/B0007_part2.csv",
    "/Users/Axel/fedn/examples/server-functions/new_data/B0046.csv",
    "/Users/Axel/fedn/examples/server-functions/new_data/B0047.csv",
    "/Users/Axel/fedn/examples/server-functions/new_data/B0048.csv"
)

In [3]:
num_clients = 9  # Total number of clients
num_partitions = 3  # Number of partitions (dist values)

# Calculate how many clients per partition
clients_per_partition = num_clients // num_partitions

# List to store the created clients
clients = []

# Loop through to create clients with a more robust partitioning logic
for idx in range(num_clients):
    cid = str(idx)
    dist = idx // clients_per_partition  # Assign partition ID (dist value) based on index

    # Create the client and generate data
    client = sim.Client_Sim(id=cid, dist=dist)

    clients.append(client)

# Print out the client ids and their associated partition (dist)
clients_sorted = sorted(clients, key=lambda c: c.dist)

# Print their IDs grouped by dist
print("Client IDs grouped by dist:")
for dist_val in sorted(set(c.dist for c in clients_sorted)):
    group = [c.id for c in clients_sorted if c.dist == dist_val]
    print(f"dist {dist_val}: {group}")


Client IDs grouped by dist:
dist 0: ['0', '1', '2']
dist 1: ['3', '4', '5']
dist 2: ['6', '7', '8']


In [4]:
## hard coded
model = mod.compile_model(8)
print(model)

BatterySoHModel(
  (fc1): Linear(in_features=8, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
)


In [5]:
num_rounds = 20
parameters_np = [param.detach().cpu().numpy() for param in model.parameters()]


In [6]:
sf = ServerFunctions()

client_ids = [c.id for c in clients]
sf.client_clusters = {cid: 0 for cid in client_ids}

#print(client_ids)

for i in range(num_rounds):
    round = i
    print('-'*15, f'PSYCHOSIS!! - starting round {round}', '-'*15)
    
    try:
        selected_ids = sf.client_selection(client_ids)
        selected_clients = [c for c in clients if c.id in selected_ids]
        #config = sf.client_settings(parameters_np)
        all_settings = sf.client_settings(parameters_np)
        # print("→ client_clusters:", sf.client_clusters)
        # print("→ all_settings keys:", all_settings.keys())
    
    except:
        print("fucked up at settings part innit?")
    
    
    
    example_mode = all_settings[selected_ids[0]]["mode"]

    #print(f"example_mode : {example_mode}")

    print(f'PSYCHOSIS MODE IS {example_mode}')

    if example_mode == "log":
        for i, cli in enumerate(selected_clients):
            try: 
                cli.gen_data(filepath=datafiles[i],current_round=round, total_rounds=num_rounds)
            except:
                print(f"fucked up generating the data ya dipshit t('_'t")
            cli.act(all_settings[cli.id], round)
        new_global = sf.aggregate(parameters_np, {})
    else:
        for_agg = {}
        for i, cli in enumerate(selected_clients):
            cfg = all_settings[cli.id]
            cli.gen_data(filepath=datafiles[i],current_round=round, total_rounds=num_rounds)
        
        
            output = cli.act(cfg, round=round)
            mode   = cfg["mode"]

            print(f"mode : {mode}, and type: {type(mode)}")
            #print(f"cfg = {cfg}")
            try:
                if isinstance(output, tuple):
                    if mode == "vanilla" and len(output) == 3:
                        updated_model, loss, val_loss = output
                    elif len(output) == 2:
                        updated_model, loss = output
                    else:
                        updated_model = output[0]
                else: 
                    updated_model = output
            except:
                print("fkn Biden administration's fault")
            
            # Convert model to List[np.ndarray]
            updated_np = [val.cpu().numpy() for _, val in updated_model.state_dict().items()]
            
            #print(f"updated_np : {updated_np}")

            # Build metadata dict
            metadata = {
                "loss": loss,
                "mode": mode,
                "distribution": cli.dist,
                "num_examples": cli.X.shape[0],
                "recent_stats": cli.stats
            }

            if mode == "vanilla" and val_loss is not None:
                print('MODE IS VANILLA!!!')
                metadata["val_loss"] = val_loss
                print(val_loss)

            # Store in aggregation dictionary
            for_agg[cli.id] = (updated_np, metadata)

        new_global = sf.aggregate(parameters_np, for_agg)

    # if sf.handler.stop:
    #     print(f'[INFO] - stopping simulation')
    #     break

    print('-'*10, 'FINISHED', '-'*10)


--------------- PSYCHOSIS!! - starting round 0 ---------------
[DEBUG] Selecting all 9 clients for the round.
Function Activated! running client_settings function
round = 0
[INFO] Sending seed model to all clients.
self.client_clusters : {'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0}
['0', '1', '2', '3', '4', '5', '6', '7', '8']
<class 'dict'>
PSYCHOSIS MODE IS vanilla
Vanilla strikes!
val loss = 8150.796875
mode : vanilla, and type: <class 'str'>
MODE IS VANILLA!!!
8150.796875
Vanilla strikes!
val loss = 6518.5185546875
mode : vanilla, and type: <class 'str'>
MODE IS VANILLA!!!
6518.5185546875
Vanilla strikes!
val loss = 9757.26953125
mode : vanilla, and type: <class 'str'>
MODE IS VANILLA!!!
9757.26953125
Vanilla strikes!
val loss = 5850.05224609375
mode : vanilla, and type: <class 'str'>
MODE IS VANILLA!!!
5850.05224609375
Vanilla strikes!
val loss = 8557.77734375
mode : vanilla, and type: <class 'str'>
MODE IS VANILLA!!!
8557.77734375
Vanilla strikes!
val 

In [7]:
print('DONE')

DONE
